In [ ]:
import tensorflow as tf
import numpy as np
import pathlib
import data

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model, Sequential
from tqdm                                       import tqdm

In [ ]:
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = sorted(modelCheckpointPath.glob("*"))
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
def getPartialModel(savedModelPath):
    loadedModel = load_model(savedModelPath)
    
    indexOfGAPLayer = -1
    # Get the index of the last GlobalAveragePooling2D layer starting from the back of the list of layers
    for layer in loadedModel.layers[::-1]:
        layer_type = str(type(layer))
        if layer_type != "<class 'tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D'>":
            indexOfGAPLayer -= 1
        else:
            break

    if indexOfGAPLayer != -1:
        indexAfterGAPLayer = indexOfGAPLayer + 1
        partialModel = Sequential(loadedModel.layers[:indexAfterGAPLayer])
    else:
        # The GlobalAveragePooling2D Layer is the last layer. Thus, we get all the layers (no need to slice the list).
        partialModel = Sequential(loadedModel.layers[:])
    
    return partialModel

In [ ]:
def extractFeatures(model, framePath):
    frame = image.load_img(framePath, 
                           target_size   = (299,299), 
                           interpolation = "lanczos") # shape: (299, 299, 3): # of dim: 3
    frame_arr = image.img_to_array(frame)             # pixel values in range [0,255]
    frame_arr = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
    frame_arr = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
    features  = model.predict(frame_arr)              # returns numpy array of shape: (1, 2048): # of dim: 2
    features  = features[0]                           # shape: (2048, ): # of dim: 1
    return features

In [ ]:
def saveFeatureSequences(model, dataObj):
    sequencesPath = pathlib.Path(r"./Sequences")
    for dataRow in tqdm(dataObj.data[:]):
        sequencePath = sequencesPath/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence")
        # get the list of paths of the frames of the video referenced in dataRow
        framePaths = dataObj.getFramesForVideo(dataRow)
        
        featureSequence = []
        for framePath in framePaths:
            features = extractFeatures(model, framePath)
            featureSequence.append(features)
        np.save(sequencePath, featureSequence)

In [ ]:
def main():
    dataObj    = data.Data()
    numClasses = dataObj.numClasses
    modelCheckpointPath = pathlib.Path(r"./Callbacks/CNN")/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath = getBestSavedModel(modelCheckpointPath)
    partialModel   = getPartialModel(savedModelPath)
    saveFeatureSequences(partialModel, dataObj)

In [ ]:
main()